In [1]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import os
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pandas as pd
import csv

from nc_processing import *
from analysis import * 

%matplotlib inline

In [2]:
# define the data directory to search for files using lists
ceda_dir='/badc/cmip6/data/CMIP6/{project}/{centre}/{model}/{exp}/{run}/{domain}/{var}/gn/latest/'
# define the base of the filename for the output file(s) using lists, note this matches the file format in the data directory except for the date_4_file section and the missing .nc. 
out_base='{var}_{domain}_{model}_{exp}_{run}_gn_{time_range}' # all that's missing is a .nc but we can add that later.

In [3]:
def global_mean(var,exp,dates,project):
    data_dir='/home/users/quigley/Projects/geoengineering_summer' #Changed from Pete's directory
    model='UKESM1-0-LL'
    centre='MOHC'
    
    if var == "evspsblpot":
        domain = "Emon"  # source documentation said it used Emon, not Amon, so I gave this a try. Didn't work
    else:
        domain='Amon' # pr is an Amon variable.
    
    runs=['r1i1p1f2','r4i1p1f2','r8i1p1f2']
    grid='gn'
    season='ANN'
    time_files=1
    
    # Use the get_fixed() function to return the gridcell area and land fraction for the model we want.
    ds_area, ds_land = get_fixed('MOHC','UKESM1-0-LL','r1i1p1f2') 

    # Let's define a data array for the area weight of the gridcells
    da_weight = ds_area['areacella'] / ds_area['areacella'].sum()
    da_weight = da_weight.rename(new_name_or_name_dict='area_weight')

    # load the arguments into a list then... 
    args=[season,dates,data_dir,model,centre,var,domain,exp,project,runs,grid,time_files]
    ds_mean, ds_std = get_ens_seasonal_mean_std(*args) # ... unpack them into the function.
    
    # multiply each temperature value with its corresponding fraction of total area, then sum to find the global mean.
    return (ds_mean[var]*da_weight).sum() - 273.15 # convert from K to C.


In [4]:
def frac_moderated_exacerbated(var):
    # Use the get_fixed() function to return the gridcell area and land fraction for the model we want.
    ds_area, ds_land = get_fixed('MOHC','UKESM1-0-LL','r1i1p1f2') 
    # NOTE - to do this for another model, you'll need to look up the run number used in the piControl experiment which stores the fx variables.
    # search on CEDA archive for "areacella" and "<MODEL NAME>" and it should show you.

    # Let's define a data array for the area weight of the gridcells
    da_weight = ds_area['areacella'] / ds_area['areacella'].sum()
    da_weight = da_weight.rename(new_name_or_name_dict='area_weight')
    # land area = gricell area * land fraction ([0-100] * 0.01)
    da_land_area = ds_area['areacella'] * ds_land['sftlf'] * 0.01 
    # Now let's create a land area weighting
    da_land_weight = da_land_area / da_land_area.sum()
    
    data_dir='/home/users/quigley/Projects/geoengineering_summer'

    # Set variables that are common to all experiments.
    model='UKESM1-0-LL'
    centre='MOHC' 
    
    if var == "evspsblpot":
        domain = "Emon"
    else:
        domain='Amon' # pr is an Amon variable.
    grid='gn'
    season='ANN'

    # specify a list of runs
    runs=['r1i1p1f2','r4i1p1f2','r8i1p1f2']

    # Load G6sulfur ensemble-mean datasets
    exp='G6sulfur'
    project='GeoMIP'
    dates=['2070-01-01','2100-01-01']
    ds_G6sulfur_mean, ds_G6sulfur_std = get_ens_seasonal_mean_std(season, dates, data_dir, model, centre, var, domain, exp, project, runs, grid, time_files=1)# setting time files to zero.

    # Load ssp585 ensemble-mean datasets
    project='ScenarioMIP'
    exp='ssp585'
    dates=['2070-01-01','2100-01-01']
    ds_ssp585_mean, ds_ssp585_std = get_ens_seasonal_mean_std(season, dates, data_dir, model, centre, var, domain, exp, project, runs, grid, time_files=1)# setting time files to zero.

    # Load ssp585 ensemble-mean datasets
    project='CMIP'
    exp='historical'
    dates=['1960-01-01','1990-01-01']
    ds_historical_mean, ds_historical_std = get_ens_seasonal_mean_std(season, dates, data_dir, model, centre, var, domain, exp, project, runs, grid, time_files=1)# setting time files to zero.
    
    # Call my better_worse_off function, passing in the data_arrays for the variable of interest rather than the full datasets.
    better, worse, dunno = better_worse_off(ds_G6sulfur_mean[var], ds_G6sulfur_std[var], ds_ssp585_mean[var], ds_ssp585_std[var], ds_historical_mean[var], ds_historical_std[var], 90, 0.05)
    
    # Calculate fraction as % by multiplying the "better" data-array (1 for better, 0 for other) by the area weight array, then summing over all points.
    global_moderated = 100.*((better + 0) * da_weight).values

    global_exacerbated = 100.*((worse + 0) * da_weight).values

    # Calculate fraction as % by multiplying the "better" data-array (1 for better, 0 for other) by the area weight array, then summing over all points.
    land_moderated = 100.*((better + 0) * da_land_weight).values

    land_exacerbated = 100.*((worse + 0) * da_land_weight).values
    
    return global_moderated, global_exacerbated, land_moderated, land_exacerbated


## Now I'm calculating whether areas experiencing significant climate change are more likely to be moderated or exacerbated vs. areas that don't.

In [5]:
#Finding which areas experience climate change
def climate_change(var):
    # Use the get_fixed() function to return the gridcell area and land fraction for the model we want.
    ds_area, ds_land = get_fixed('MOHC','UKESM1-0-LL','r1i1p1f2') 
    # NOTE - to do this for another model, you'll need to look up the run number used in the piControl experiment which stores the fx variables.
    # search on CEDA archive for "areacella" and "<MODEL NAME>" and it should show you.

    # Let's define a data array for the area weight of the gridcells
    da_weight = ds_area['areacella'] / ds_area['areacella'].sum()
    da_weight = da_weight.rename(new_name_or_name_dict='area_weight')
    # land area = gricell area * land fraction ([0-100] * 0.01)
    da_land_area = ds_area['areacella'] * ds_land['sftlf'] * 0.01 
    # Now let's create a land area weighting
    da_land_weight = da_land_area / da_land_area.sum()
    
    data_dir='/home/users/quigley/Projects/geoengineering_summer'

    # Set variables that are common to all experiments.
    model='UKESM1-0-LL'
    centre='MOHC' 
    
    domain='Amon' # pr is an Amon variable.
    grid='gn'
    season='ANN'

    # specify a list of runs
    runs=['r1i1p1f2','r4i1p1f2','r8i1p1f2']
    num_years = 90

    # Load ssp585 ensemble-mean datasets
    project='ScenarioMIP'
    exp='ssp585'
    dates=['2070-01-01','2100-01-01']
    ds_ssp585_mean, ds_ssp585_std = get_ens_seasonal_mean_std(season, dates, data_dir, model, centre, var, domain, exp, project, runs, grid, time_files=1)# setting time files to zero.

    # Load ssp585 ensemble-mean datasets
    project='CMIP'
    exp='historical'
    dates=['1960-01-01','1990-01-01']
    ds_historical_mean, ds_historical_std = get_ens_seasonal_mean_std(season, dates, data_dir, model, centre, var, domain, exp, project, runs, grid, time_files=1)# setting time files to zero.

    # ttest_sub returns a numpy array of P-values, where P is between 0 and 1. for 95% significance P is below 0.05
    ttest_pvalue = ttest_sub(ds_ssp585_mean[var],ds_ssp585_std[var],num_years,ds_historical_mean[var],ds_historical_std[var],num_years)
    
    longitude = len(ttest_pvalue[:,0])
    latitude = len(ttest_pvalue[0,:])
    
    climate_changed_areas = np.empty([longitude,latitude])
    
    for i in range(0, longitude-1):
        for j in range(0, latitude-1):
            #print(ttest_pvalue[i,j])
            if ttest_pvalue[i,j] < 0.05:
                climate_changed_areas[i,j] = 1
            else:
                climate_changed_areas[i,j] = 0
    
    # Calculate fraction as % by multiplying the "better" data-array (1 for better, 0 for other) by the area weight array, then summing over all points.
    global_changed = 100.*((climate_changed_areas) * da_weight).values

    # Calculate fraction as % by multiplying the "better" data-array (1 for better, 0 for other) by the area weight array, then summing over all points.
    land_changed = 100.*((climate_changed_areas) * da_land_weight).values
 
    return global_changed, land_changed, climate_changed_areas


In [6]:
def moderated_exacerbated(var):
    # Use the get_fixed() function to return the gridcell area and land fraction for the model we want.
    ds_area, ds_land = get_fixed('MOHC','UKESM1-0-LL','r1i1p1f2') 
    # NOTE - to do this for another model, you'll need to look up the run number used in the piControl experiment which stores the fx variables.
    # search on CEDA archive for "areacella" and "<MODEL NAME>" and it should show you.
    
    data_dir='/home/users/quigley/Projects/geoengineering_summer'

    # Set variables that are common to all experiments.
    model='UKESM1-0-LL'
    centre='MOHC' 
    
    if var == "evspsblpot":
        domain = "Emon"
    else:
        domain='Amon' # pr is an Amon variable.
    grid='gn'
    season='ANN'

    # specify a list of runs
    runs=['r1i1p1f2','r4i1p1f2','r8i1p1f2']

    # Load G6sulfur ensemble-mean datasets
    exp='G6sulfur'
    project='GeoMIP'
    dates=['2070-01-01','2100-01-01']
    ds_G6sulfur_mean, ds_G6sulfur_std = get_ens_seasonal_mean_std(season, dates, data_dir, model, centre, var, domain, exp, project, runs, grid, time_files=1)# setting time files to zero.

    # Load ssp585 ensemble-mean datasets
    project='ScenarioMIP'
    exp='ssp585'
    dates=['2070-01-01','2100-01-01']
    ds_ssp585_mean, ds_ssp585_std = get_ens_seasonal_mean_std(season, dates, data_dir, model, centre, var, domain, exp, project, runs, grid, time_files=1)# setting time files to zero.

    # Load ssp585 ensemble-mean datasets
    project='CMIP'
    exp='historical'
    dates=['1960-01-01','1990-01-01']
    ds_historical_mean, ds_historical_std = get_ens_seasonal_mean_std(season, dates, data_dir, model, centre, var, domain, exp, project, runs, grid, time_files=1)# setting time files to zero.
    
    # Call my better_worse_off function, passing in the data_arrays for the variable of interest rather than the full datasets.
    better, worse, dunno = better_worse_off(ds_G6sulfur_mean[var], ds_G6sulfur_std[var], ds_ssp585_mean[var], ds_ssp585_std[var], ds_historical_mean[var], ds_historical_std[var], 90, 0.05)
    
    # numpy array not weighted by area or landmass
    moderated = (better + 0).values

    exacerbated = (worse + 0).values
    
    return moderated, exacerbated


In [7]:
def change_mod_exac(var):
    
    # Use the get_fixed() function to return the gridcell area and land fraction for the model we want.
    ds_area, ds_land = get_fixed('MOHC','UKESM1-0-LL','r1i1p1f2')

    # Let's define a data array for the area weight of the gridcells
    da_weight = ds_area['areacella'] / ds_area['areacella'].sum()
    da_weight = da_weight.rename(new_name_or_name_dict='area_weight')
    # land area = gricell area * land fraction ([0-100] * 0.01)
    da_land_area = ds_area['areacella'] * ds_land['sftlf'] * 0.01 
    # Now let's create a land area weighting
    da_land_weight = da_land_area / da_land_area.sum()
    
    moderated, exacerbated = moderated_exacerbated(var)
    global_changed, land_changed, climate_changed_areas = climate_change(var)

    longitude = len(climate_changed_areas[:,0]) #determining length and width (cell number) of model grid
    latitude = len(climate_changed_areas[0,:])

    changed_moderated = np.empty([longitude,latitude]) #Creating empty numpy arrays the size of the grid
    changed_exacerbated = np.empty([longitude,latitude])
    unchanged_moderated = np.empty([longitude,latitude])
    unchanged_exacerbated = np.empty([longitude,latitude])

    for i in range(0, longitude-1):
        for j in range(0, latitude-1):
            if climate_changed_areas[i,j] == 1: #Finding areas that experience significant climate change
                unchanged_moderated[i,j] = 0
                unchanged_exacerbated[i,j] = 0
                if moderated[i,j] == 1:         #Finding moderated areas w/ sig climate change
                    changed_moderated[i,j] = 1
                    changed_exacerbated[i,j] = 0
                elif exacerbated[i,j] == 1:     #Finding exacerbated areas w/ sig climate change
                    changed_moderated[i,j] = 0
                    changed_exacerbated[i,j] = 1
                
            else:                              #Areas that don't experience significant climate change
                changed_moderated[i,j] = 0
                changed_exacerbated[i,j] = 0
                if moderated[i,j] == 1:         #Finding moderated areas w/out sig climate change
                    changed_moderated[i,j] = 1
                    changed_exacerbated[i,j] = 0
                elif exacerbated[i,j] == 1:     #Finding exacerbated areas w/out sig climate change
                    changed_moderated[i,j] = 0
                    changed_exacerbated[i,j] = 1

    #c=changed,u=unchanged                  the names were just getting a bit long
    #m=moderated,e=exacerbated
    #g=global,l=land

    cmg = (100*changed_moderated*da_weight).sum().values
    ceg = (100*changed_exacerbated*da_weight).sum().values
    cml = (100*changed_moderated*da_land_weight).sum().values
    cel = (100*changed_exacerbated*da_land_weight).sum().values

    umg = (100*unchanged_moderated*da_weight).sum().values
    ueg = (100*unchanged_exacerbated*da_weight).sum().values
    uml = (100*unchanged_moderated*da_land_weight).sum().values
    uel = (100*unchanged_exacerbated*da_land_weight).sum().values
    
    return cmg,ceg,cml,cel,umg,ueg,uml,uel

In [8]:
def calculate(var, metric):
    #Using.values converts xarray to numpy array
    control_mean=global_mean(var,'historical',['1960-01-01','1990-01-01'],'CMIP').values
    geo_mean=global_mean(var,'G6sulfur',['2070-01-01','2100-01-01'],'GeoMIP').values  
    solar_mean=global_mean(var,'G6solar',['2070-01-01','2100-01-01'],'GeoMIP').values
    sulfur_mean=global_mean(var,'G6sulfur',['2070-01-01','2100-01-01'],'GeoMIP').values
    warming_mean=global_mean(var,"ssp585",['2070-01-01','2100-01-01'],"ScenarioMIP").values
    
    geo_anom = geo_mean - control_mean
    sulfur_solar_anom = sulfur_mean - solar_mean
    geo_warming_anom = geo_mean - warming_mean
    warming_anom = warming_mean - control_mean
    
    global_moderated, global_exacerbated, land_moderated, land_exacerbated = frac_moderated_exacerbated(var)
    cmg,ceg,cml,cel,umg,ueg,uml,uel = change_mod_exac(var)
    
    if metric == "global_mean_anomaly_control":
        return geo_anom
    elif metric == "global_mean_anomaly_solar":
        return sulfur_solar_anom
    elif metric == "global_mean_anomaly_warming":
        return geo_warming_anom
    elif metric == "moderated_exacerbated":
        if geo_anom < warming_anom:
            return "moderated"
        elif warming_anom < geo_anom:
            return "exacerbated"
        else:
            return "N/A"
    elif metric == "global_moderated":
        return global_moderated.sum()
    elif metric == "global_exacerbated":
        return global_exacerbated.sum()
    elif metric == "land_moderated":
        return land_moderated.sum()
    elif metric == "land_exacerbated":
        return land_exacerbated.sum()
    elif metric == "changed_mod_global":
        return cmg
    elif metric == "changed_exac_global":
        return ceg
    elif metric == "changed_mod_land":
        return cml
    elif metric == "changed_exac_land":
        return cel
    elif metric == "unchanged_mod_global":
        return umg
    elif metric == "unchanged_exac_global":
        return ueg
    elif metric == "unchanged_mod_land":
        return uml
    elif metric == "unchanged_exac_land":
        return uel
    else:
        return 0

In [9]:
var_list = ['tas',"tasmax","tasmin","pr","huss"] #"evspsblpot","eow","tos","prhmax"]
metric_list = ['global_mean_anomaly_control',"global_mean_anomaly_solar","global_mean_anomaly_warming","moderated_exacerbated","global_moderated","global_exacerbated","land_moderated","land_exacerbated","changed_mod_global","changed_exac_global","changed_mod_land","changed_exac_land","unchanged_mod_global","unchanged_exac_global","unchanged_mod_land","unchanged_exac_land"]

dict_outer = {}
for var in var_list:
    dict_inner = {}
    for metric in metric_list:
        dict_inner[metric] = calculate(var,metric)       #links output to metric
    dict_outer[var] = dict_inner                         #links output to variable
    #end for metric
#end for var

df = pd.DataFrame.from_dict(dict_outer)
df.to_csv('metric_table.csv')

loading existing files tas_Amon_UKESM1-0-LL_historical_ens-mean_gn_1960-01-01_1990-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_G6sulfur_ens-mean_gn_2070-01-01_2100-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_G6solar_ens-mean_gn_2070-01-01_2100-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_G6sulfur_ens-mean_gn_2070-01-01_2100-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_ssp585_ens-mean_gn_2070-01-01_2100-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_G6sulfur_ens-mean_gn_2070-01-01_2100-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_ssp585_ens-mean_gn_2070-01-01_2100-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_historical_ens-mean_gn_1960-01-01_1990-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_G6sulfur_ens-mean_gn_2070-01-01_2100-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_ssp585_ens-mean_gn_2070-01-01_2100-01-01 ANN
loading existing files tas_Amon_UKESM1-0-LL_historical_ens-mean_gn_1960-01-01_1990-01-01 ANN
